# Nerual Network for Classifying fine-grained images

In [6]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import pandas as pd
import os
from libcpab.libcpab.pytorch import cpab
from libcpab.libcpab.helper.utility import show_images
from IPython.display import display
import random
from sklearn.metrics import accuracy_score
import pylab as pl
from IPython import display
from skimage import io
from skimage.transform import rescale
import datetime
import math
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.models as models

----------------------------------------------------------------------
Operating system: darwin
Using pytorch backend
----------------------------------------------------------------------


# Dataloader test

In [7]:
class Custom_Loss(nn.modules.Module):
    def __init__(self):
        super(Custom_Loss, self).__init__()
        self.N = trainloader.onehot.shape[0]
        self.Nm = np.array(trainloader.countlist)
        self.W0 = [max( x /self.N,0.1) for x in self.Nm]
        self.W = [(1-w0)/w0 for w0 in self.W0]
        
    def forward(self, input, target):
        N = input.shape[0]
        M = target.shape[1]
        T = torch.tensor(self.W)
        T = T.type(input.type())
        result = torch.tensor(0)
        result = result.type(output.type())
        
        for n in range(N):
            #result += sum((T*target[n]*torch.log10(abs(input[n])+1e-16)+(1-target[n])*torch.log10(1-abs(input[n]))))
            result += -1*sum(T*target[n]*(torch.log_softmax(input[n],dim=0))+(1-target[n])*(torch.log_softmax(1-input[n],dim=0)))
                
        result = (1/(N*M))*result
       
        return result

In [8]:
def calculate_precision(pred, true):
    predicted_positives = (np.array(pred) == 1).sum()
    
    true_positives = np.array([a == b and a == 1 for a, b in zip(pred, true)])
    true_positives = (np.array(true_positives) == 1).sum()

    result = true_positives / predicted_positives
    
    return result

def AP(pred, true):
    precision = calculate_precision(pred, true)
    a = np.array([a == b and a == 1 for a, b in zip(pred, true)])
    score = (np.array(a) ==True).sum() * precision
    
    result = score / K
    
    return result



def MAP(predicted_data, true_data):
    result = np.array([AP(pred, true) for pred, true in zip(predicted_data, true_data)]).sum() / K

    return result

In [9]:
def upsample(path=None, samples=None, DIM=224, optimize=False, N_fixed=None, N_opt=2, show_result=False):
    """
    Upsample images with CPAB augmentation technique.
    Augmented images are saved in same directory as original images.

        Args
            path: Path to directory with images.
            samples: Dictionary with ID's and corresponding upsampling size. Ex: {'ID_1': 10}
            DIM: Dimension of agumented images. (integer as H x W are the same)
            optimize: If true the augmentation is optimized with the Adam optimizer,
                      and the learned estimation of the transformed data is returned
                      instead of a first transformation. Default false.
            N_fixed: Number of augmented samples to return, should be give in samples dict {'sample_id': N}.
            N_opt: Number of times optimization procedure should be run, default 2.
            show_result: If true transformed images are presented, default false.
    """

    if not path:
        print("Please specify path to directory with samples.")
        return 0

    if not samples:
        print("No dictionary with sample ID's and sample size given. See _calculate_augmentation()")
        return 0

    # convert images from PNG to JPG
    for sample in samples:
        # number of times sample should be upsampled
        N = samples.get(sample)
        if N_fixed != None:
            N = N_fixed

        # read image and convert from RGBA to RGB
        im = Image.open(path+sample)

        # convert images from RGBA to RGB
        if im.mode == "RGBA":
            im.load() # required for im.split()
            newIm = Image.new("RGB", im.size, color=(0, 0, 0)) # black - RGB(0,0,0) - background color
            newIm.paste(im, mask=im.split()[3]) # 3 is the alpha channel
            newIm.save(path+sample, "PNG", quality=80) # save image with old file ending name
        else:
            pass

        # augment images by estimation of the transformed data
        if optimize:
            for n in range(N):
                # read image again as RGB with 3 channels and normalize between [0, 1]
                data = plt.imread(path+sample, format="RGB") / 255
                data = np.expand_dims(data, 0) # create batch of data
                # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
                data = torch.Tensor(data).permute(0,3,1,2)

                # Define transformer class
                T1 = cpab(tess_size=[3,3], device='cpu')

                # Sample random transformation
                theta_true = 0.5*T1.sample_transformation(1)

                # Transform the images
                transformed_data = T1.transform_data(data, theta_true, outsize=(DIM,DIM))

                # define a PyTorch procedure that enables estimation of the transformation
                T2 = cpab(tess_size=[3,3], device='cpu')
                theta_est = T2.identity(1, epsilon=1e-4)
                theta_est.requires_grad = True

                # PyTorch optimizer for estimation procedure
                optimizer = torch.optim.Adam([theta_est], lr=0.1)

                # Optimization loop
                max_opt = N_opt
                for i in range(max_opt):
                    trans_est = T2.transform_data(data, theta_est, outsize=(DIM, DIM))
                    loss = (transformed_data.to(trans_est.device) - trans_est).pow(2).mean()
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                if show_result:
                    plt.subplots(1,3, figsize=(10, 15))
                    plt.subplot(1,3,1)
                    plt.imshow(data.permute(0,2,3,1).numpy()[0])
                    plt.axis('off')
                    plt.title('Source')
                    plt.subplot(1,3,2)
                    plt.imshow(transformed_data.permute(0,2,3,1).cpu().numpy()[0])
                    plt.axis('off')
                    plt.title('Target')
                    plt.subplot(1,3,3)
                    plt.imshow(trans_est.permute(0,2,3,1).cpu().detach().numpy()[0])
                    plt.axis('off')
                    plt.title('Estimate')
                    plt.show()

                # Get the corresponding numpy arrays in correct format [n_batch, width, height, n_channels]
                transformed_data = trans_est.permute(0, 2, 3, 1).cpu().detach().numpy()[0]

                # save augmented versions
                # name of image: aug[x]_[original_name] where x is in the interval [0,N]
                im = transformed_data * 255 # normalize pixel values between [0,255]
                filename = "aug"+str(n)+"_"+sample
                image = Image.fromarray(im.astype('uint8'), mode="RGB")
                image.save(os.path.join(path, filename)) # save in class folder

        # augment images without transformation optimization
        else:
            # read image again as RGB with 3 channels and normalize between [0, 1]
            data = plt.imread(path+sample, format="RGB") / 255
            data = np.tile(np.expand_dims(data, 0), [N,1,1,1]) # create batch of data
            # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
            data = torch.Tensor(data).permute(0,3,1,2)

            # Define transformer class
            T1 = cpab(tess_size=[3,3], device='cpu')

            # Sample random transformation
            theta_true = 0.5*T1.sample_transformation(N)

            # Transform the images
            transformed_data = T1.transform_data(data, theta_true, outsize=(DIM,DIM))

            # Get the corresponding numpy arrays in correct format [n_batch, width, height, n_channels]
            transformed_data = transformed_data.permute(0, 2, 3, 1).cpu().numpy()

            if show_result:
                show_images(transformed_data)

            # save augmented versions
            # name of image: aug[x]_[original_name] where x is in the interval [0,N]
            n = 0
            for data_sample in transformed_data:
                im = data_sample * 255 # normalize pixel values between [0,255]
                filename = "aug"+str(n)+"_"+sample
                image = Image.fromarray(im.astype('uint8'), mode="RGB")
                image.save(os.path.join(path, filename)) # save in class folder
                n += 1
    return

In [24]:
def calculate_augmentation(df, criteria=1000, restriction=10):
    """
    Calculate how many times a sample is to be upsampled with augmentation
    compared to the represantation of its class or if multilabel, its classes.

    Be aware of samples of a low represented class can be part of a larger class also.
    The aforementioned can result in upsampling of high represented classes as well,
    when upsampling low represented classes.

    Args
        df: A dataframe which the upsampling size is calculated from.
            The indexes should be the ID's of the samples.
        criteria: The criteria to match number of times to upsample a class with.
    """
    # placeholders
    classes = {}
    samplesToAugment = {}

    # get number of occurrences for each class
    for column in df.columns:
        num_occ = len(df.loc[df[column] == 1])
        classes[column] = num_occ

    # sort classes by ascending size
    classes_sorted = sorted(classes.items(), key=lambda value: value[1])

    # iterate over classes and calculate
    # number of times to augment data to
    # match specified criteria
    for cls in classes_sorted:
        # get indexes for samples where class value is 1
        indexesInClass = df.loc[df[cls[0]] == 1].index

        # remove indexes that already are appended to samplesToAugment
        indexesToAugment = [indexesInClass[i] for i in range(len(indexesInClass)) \
                            if indexesInClass[i] not in samplesToAugment.keys()]

        # find how many times already is upsampled
        already_size = sum([samplesToAugment.get(sample) for sample in indexesInClass \
                            if sample in samplesToAugment.keys() and sample not in indexesToAugment])

        # calculate number of times samples in class 
        # should be upsampled to match specified criteria
        if len(indexesToAugment) == 0:
            augment_size = int(1000 - already_size)
        # split upsample size over all samples to augment
        else:
            augment_size = int((1000-already_size) / len(indexesToAugment))

        # respect restriction criteria
        if augment_size > restriction:
            augment_size = 10
        # if class is already upsampled enough
        # do not upsample remaining samples in class
        elif augment_size < 0:
            augment_size = 0

        # save indexes to augment and corresponding number of times to upsample
        for index in indexesToAugment:
            if index in samplesToAugment.keys():
                print("Index already in samplesToAugment, overwriting key..")
            samplesToAugment[index] = augment_size

    return samplesToAugment

In [11]:
def get_onehot_augmented(path=None, search_string='aug'):
    """
    Designed for appending augmented data to original dataframe.
    Append data to a dataframe where appended data gets its values from
    its sample of origin in the dataframe. 

    Args
        df: Pandas dataframe which data is to be appended to.
        path: Path to directory with augmented files.
        search_string: A regex string criteria for searching for files in directory.
    """
    if not path:
        print("Specify the path to augmented files.")
        return 0

    # get filenames for augmented images
    filenames = [name for name in os.listdir(path) if os.path.isfile(path+name) and search_string in name]

    # construct temporary dataframe for data to append
    df_append = pd.DataFrame(data=[name for name in filenames], columns=['id']).set_index(keys='id')

    # iterate over all row indexes in dataframe to append
    for i in df_append.index:
        # get original target values for original image for augmented image
        values = df.iloc[list(np.where(df.index == i.replace('aug_', ''))[0])].values

        # iterate over columns and corresponding values in dataframe to append to
        for column, value in zip(range(len(df.columns)), values[0]):
            df_append[f"{df.columns[column]}"] = value

    df_new = pd.concat([self.df, df_append])

    return df_new

In [ ]:
def train_valid_split(df, trainsplit_size=0.8):
    """
    Split a dataset in training- and validation split.

    Split is made by splitting the least represented class first
    and the highest represented class last.

    Function is made for multilabel problems where samples that are already 
    drawn from a previous class are not drawn again, and the ratio of the 
    split is calculated at each class from the previous draws to match the 
    split criteria.

    Args
        df: A dataframe which the split should be created from.
            The indexes should be the ID's of the samples.
        trainsplit_size: The size of the training split. 
                         Should be a float in the interval [0, 1]
    """
    # placeholders
    cls = {}
    train_idx = []
    valid_idx = []

    # get number of occurrences for each class
    for col in df.columns:
        size = len(df.loc[df[col] == 1])
        cls[col] = size

    # sort classes by ascending
    cls_sorted = sorted(cls.items(), key=lambda value: value[1])

    # iterate over classes from least represented class
    # and draw samples for training- and validation split
    for col in cls_sorted:
        # get indexes for samples where class value is 1
        indexes = df.loc[df[col[0]] == 1].index

        # Remove indexes that already are appended to test_idx array
        indexes = [indexes[i] for i in range(len(indexes)) if indexes[i] not in valid_idx and indexes[i] not in train_idx]

        # get size of how many indexes should be drawn for train
        train_size = int(len(indexes) * trainsplit_size)

        # get indexes for train- and validation split
        idx_train = [indexes[i] for i in sorted(random.sample(range(len(indexes)), train_size))]
        idx_valid = [indexes[i] for i in range(len(indexes)) if indexes[i] not in idx_train]

        # save indexes
        train_idx.extend(idx_train)
        valid_idx.extend(idx_valid)

    return train_idx, valid_idx

In [20]:
class Trainloader:
    
    def __init__(self, croppedfolder, picturesize, batchsize, csv_path, trainsplit_size):
        self.cropdirectory = croppedfolder
        self.picturesize = picturesize
        self.batchsize = batchsize
        self.onehot = self._get_onehot(csv_path)
        self.trainsplit, self.validsplit = self.train_valid_split(self.onehot, trainsplit_size)
        self.batch = 0
        
    
    def _get_onehot(self, csv_path):
        """
        Construct dataframe.

        Args
            csv_path: Path to csv file.
        """
        # construct dataframe with onehot notation
        df = pd.get_dummies(pd.read_csv(csv_path))
        # create ID column with image file names
        df['id'] = df['image_id'].apply(lambda x: str(x)) + "_" + df['tag_id'].apply(lambda x: str(x)) + ".png"
        # set ID column to index in dataframe
        df = df.set_index(keys='id')
        # only use class columns
        df = df.iloc[:, 10:]
        return df
    
    def train_valid_split(self, df, trainsplit_size=0.8):
        """
        Split a dataset in training- and validation split.

        Split is made by splitting the least represented class first
        and the highest represented class last.

        Function is made for multilabel problems where samples that are already 
        drawn from a previous class are not drawn again, and the ratio of the 
        split is calculated at each class from the previous draws to match the 
        split criteria.

        Args
            df: A dataframe which the split should be created from.
                The indexes should be the ID's of the samples.
            trainsplit_size: The size of the training split. 
                             Should be a float in the interval [0, 1]
        """
        # placeholders
        cls = {}
        train_idx = []
        valid_idx = []

        # get number of occurrences for each class
        for col in df.columns:
            size = len(df.loc[df[col] == 1])
            cls[col] = size

        # sort classes by ascending
        cls_sorted = sorted(cls.items(), key=lambda value: value[1])

        # iterate over classes from least represented class
        # and draw samples for training- and validation split
        for col in cls_sorted:
            # get indexes for samples where class value is 1
            indexes = df.loc[df[col[0]] == 1].index

            # Remove indexes that already are appended to test_idx array
            indexes = [indexes[i] for i in range(len(indexes)) if indexes[i] not in valid_idx and indexes[i] not in train_idx]

            # get size of how many indexes should be drawn for train
            train_size = int(len(indexes) * trainsplit_size)

            # get indexes for train- and validation split
            idx_train = [indexes[i] for i in sorted(random.sample(range(len(indexes)), train_size))]
            idx_valid = [indexes[i] for i in range(len(indexes)) if indexes[i] not in idx_train]

            # save indexes
            train_idx.extend(idx_train)
            valid_idx.extend(idx_valid)

        return train_idx, valid_idx

    def _convert_picture(self, path):
        image = io.imread(path)
        rescale_factor = self.picturesize / image.shape[0],self.picturesize / image.shape[1]
        scaled = rescale(image, rescale_factor, anti_aliasing=False, multichannel=True,mode='constant')[:,:,:3]
        scaled = scaled.transpose((-1, 0, 1))

        return scaled
    
    def _get_pictures(self, paths):
        trainlist = []
        for path in paths:
            trainlist.append(self._convert_picture(self.cropdirectory+path))
            
        return trainlist
    
    def train_next(self):
        try:
            paths = self.trainsplit[self.batch:self.batchsize]
            pictures = self._get_pictures(paths)
            targets = self.onehot.loc[paths].values
        
            self.batch += self.batchsize
        
            return pictures, targets
        
        except:
            self.batch = 0
            return None, None
    
    def valid_next(self):
        try:
            paths = self.validsplit[self.batch:self.batchsize]
            pictures = self._get_pictures(paths)
            targets = self.onehot.loc[paths].values
        
            self.batch += self.batchsize
        
            return pictures, targets
        
        except:
            self.batch = 0
            return None, None
        

In [ ]:
def prepare_data(batch=None,labels=None):
    if batch != None:
        # prepare data
        batch = torch.tensor(batch).float()
        labels = torch.tensor(labels).float()
        device = torch.device("cuda:0" if cuda else "cpu")
        batch = batch.to(device)
        labels = labels.to(device)

        return batch,labels
    else:
        return None, None

In [26]:
croppedfolder = "/Users/mattias/Desktop/cropped/"
csv_path = "./data/dataset_v2/train.csv"
picturesize = 224
batchsize = 64
split_ratio = 0.8 # size of training set

In [27]:
trainloader = Trainloader(croppedfolder,picturesize,batchsize,csv_path,split_ratio)

# Augment samples

### Find number of times each sample should be upsampled

In [28]:
def calculate_augmentation(df, criteria=1000, restriction=10):
    """
    Calculate how many times a sample is to be upsampled with augmentation
    compared to the represantation of its class or if multilabel, its classes.

    Be aware of samples of a low represented class can be part of a larger class also.
    The aforementioned can result in upsampling of high represented classes as well,
    when upsampling low represented classes.

    Args
        df: A dataframe which the upsampling size is calculated from.
            The indexes should be the ID's of the samples.
        criteria: The criteria to match number of times to upsample a class with.
    """
    # placeholders
    classes = {}
    samplesToAugment = {}

    # get number of occurrences for each class
    for column in df.columns:
        num_occ = len(df.loc[df[column] == 1])
        classes[column] = num_occ

    # sort classes by ascending size
    classes_sorted = sorted(classes.items(), key=lambda value: value[1])

    # iterate over classes and calculate
    # number of times to augment data to
    # match specified criteria
    for cls in classes_sorted:
        # get indexes for samples where class value is 1
        indexesInClass = df.loc[df[cls[0]] == 1].index

        # remove indexes that already are appended to samplesToAugment
        indexesToAugment = [indexesInClass[i] for i in range(len(indexesInClass)) \
                            if indexesInClass[i] not in samplesToAugment.keys()]

        # find how many times already is upsampled
        already_size = sum([samplesToAugment.get(sample) for sample in indexesInClass \
                            if sample in samplesToAugment.keys() and sample not in indexesToAugment])

        # calculate number of times samples in class 
        # should be upsampled to match specified criteria
        if len(indexesToAugment) == 0:
            augment_size = int(1000 - already_size)
        # split upsample size over all samples to augment
        else:
            augment_size = int((1000-already_size) / len(indexesToAugment))

        # respect restriction criteria
        if augment_size > restriction:
            augment_size = 10
        # if class is already upsampled enough
        # do not upsample remaining samples in class
        elif augment_size < 0:
            augment_size = 0

        # save indexes to augment and corresponding number of times to upsample
        for index in indexesToAugment:
            if index in samplesToAugment.keys():
                print("Index already in samplesToAugment, overwriting key..")
            samplesToAugment[index] = augment_size

    return samplesToAugment

samples_augment = calculate_augmentation(df=trainloader.onehot)

### Upsample the dataset using augmentation

In [29]:
upsample(path=croppedfolder, samples=samples_augment)

RuntimeError: number of dims don't match in permute

### Append the augmented samples to dataframe

In [ ]:
df_aug = get_onehot_augmented()

## Define model

In [ ]:
# Define the model
model_conv = models.resnet50(pretrained='imagenet')

# Disable autograd for resnet
for param in model_conv.parameters():
    param.requires_grad = False
    
# Change fully connected layer to match paper (autograd is default on new layers)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 1024)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_conv = model_conv.to(device)

In [ ]:
trainsplit, validsplit = train_valid_split()
trainloader = Trainloader(classfolder,croppedfolder,picturesize,batchsize)
flattened = [val for sublist in trainloader.validsplit for val in sublist]

dataset_positives = 0.0
for column in trainloader.onehot.loc[flattened].columns:
    dataset_positives += trainloader.onehot.loc[trainloader.onehot[column] == 1].sum()[column]

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        output_nodes = trainloader.onehot.shape[1]
        self.output = nn.Linear(1024,output_nodes)
       

    def forward(self, x):
        x = model_conv(x)
        x = F.relu(x)
        x = self.output(x)
        x = torch.sigmoid(x)
        return x


net = Net()

cuda = torch.cuda.is_available()
if cuda:
    net.cuda()

In [ ]:
%matplotlib inline
epochs = 100
threshold = 0.5
K = len(trainloader.classlist)
valid_every = 2
#criterion = nn.BCELoss()
#criterion = Custom_Loss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) 

losses_train = pd.DataFrame(columns=['Epoch','Loss'])
losses_valid = pd.DataFrame(columns=['Epoch','Loss'])

accuracy = 0.01 #dummy value

now = datetime.datetime.now()

for epoch in range(epochs):
    batch_loss = []
    batch,labels = trainloader.train_next()
    net.train()
    
    while batch != None:
        # Prepare data
        batch,labels = prepare_data(batch,labels)
            
        # Zero the parameter gradients
        optimizer.zero_grad()
            
        # Make prediction and backpropogate
        output = net(batch)
        loss = criterion(output,labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        
        # Load next batch
        batch,labels = trainloader.train_next()
        
    # calculate the loss of the training set   
    losses_train.loc[epoch] = [epoch+1,np.mean(batch_loss)]
    
    if epoch%valid_every == 0 or epoch == max(range(epochs)):
        batch_loss = []
        preds = []
        trues = []
        batch,labels = trainloader.valid_next()
        net.eval()
        
        while batch != None:
            # Prepare data
            batch,labels = prepare_data(batch,labels)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Make prediction
            output = net(batch)
            loss = criterion(output,labels)
            batch_loss.append(loss.item())
            
            # Append values for later calculation of accuracy
            preds.append([[1 if pred > threshold else 0 for pred in sample] for sample in output])
            trues.append([[int(value) for value in valuelist] for valuelist in labels.tolist()])
            
            # Load next batch
            batch,labels = trainloader.valid_next()
        
        # Calculate loss and accuracy for the validation set    
        losses_valid.loc[epoch] = [epoch+1,np.mean(batch_loss)]
        accuracy = MAP(preds[0], trues[0])
    
    # Plot the loss every epoch
    pl.figure(figsize=(15,5))
    pl.xlabel('Epochs')
    pl.ylabel('Loss')
    pl.title('Epoch #{}'.format(epoch+1))
    pl.plot(losses_train['Epoch'],losses_train['Loss'], '-b', label='Train')
    pl.plot(losses_valid['Epoch'],losses_valid['Loss'], '-r', label='Valid')
    pl.legend()
    pl.show()
    print('Training loss: {:.2f}'.format(losses_train['Loss'].iloc[-1],))
    print('Validation loss:{:.2f}\tValidation MAP: {:.2f}'.format(losses_valid['Loss'].iloc[-1],accuracy))
    display.clear_output(wait=True)
    

# Final plot    
pl.figure(figsize=(15,5))
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.title('Epoch #{}'.format(epoch+1))
pl.plot(losses_train['Epoch'],losses_train['Loss'], '-b', label='Train')
pl.plot(losses_valid['Epoch'],losses_valid['Loss'], '-r', label='Valid')
pl.legend()
pl.show()

print('Training loss: {:.2f}'.format(losses_train['Loss'].iloc[-1],))
print('Validation loss:{:.2f}\tValidation MAP: {:.2f}'.format(losses_valid['Loss'].iloc[-1],accuracy))

print("\n\t\t\t\t\tTraining time {}".format(datetime.datetime.now()-now))